In [13]:
import tensorflow as tf
# 手写数字相关工具包
from tensorflow.examples.tutorials.mnist import input_data
import os
import numpy as np

In [14]:
# 加载数据
# # 数据路径(无则创建)  转为one_hot(某一位数字为1，其余数字都为0)格式
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels
X = tf.placeholder('float', [None, 784])
Y = tf.placeholder('float', [None, 10])

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [15]:
# 定义权重函数
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev = 0.01))
# 初始化权重参数
w_h = init_weights([784, 625])
w_h2 = init_weights([625, 625])
w_o = init_weights([625, 10])

In [16]:
# 定义模型
def model(X, w_h, w_h2, w_o, p_keep_input, p_keep_hidden):
    # 第一个全连接层
    X = tf.nn.dropout(X, p_keep_input)
    h = tf.nn.relu(tf.matmul(X, w_h))
    h = tf.nn.dropout(h, p_keep_hidden)
    h2 = tf.nn.relu(tf.matmul(h, w_h2))
    h2 = tf.nn.dropout(h2, p_keep_hidden)
    # 输出预测值
    return tf.matmul(h2, w_o)

In [17]:
# 生成网络模型，得到预测值
p_keep_input = tf.placeholder("float")
p_keep_hidden = tf.placeholder("float")
py_x = model(X, w_h, w_h2, w_o, p_keep_input, p_keep_hidden)

In [18]:
# 定义损失函数
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x, labels=Y))
train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
predict_op = tf.argmax(py_x, 1)

In [19]:
# 将每一轮训练得到的参数储存
ckpt_dir = './ckpt_dir'
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)
# 定义一个计数器，统计训练轮数
# 计数器变量，设置它的trainable = False => 不需要训练
global_step = tf.Variable(0, name='global_step', trainable=False)

In [20]:
# 当定义完所有变量后，调用 tf.train.Saver()来保存和提取变量，其后面定义的变量将不会被存储
# 在声明完所有变量后，调用 tf.train.Saver
saver = tf.train.Saver()
# 位于 tf.train.Saver 之后的变量将不会被存储
non_storable_variable = tf.Variable(777)

In [21]:
# 训练模型并存储
with tf.Session() as sess:
    # 初始化变量
    sess.run(tf.global_variables_initializer())
    # 得到global_step的初始值
    start = global_step.eval()
    print('Start from: ', start)
    
    for i in range(start, 10):
        # 以128作为barch_size
        for start, end in zip(range(0, len(trX), 128), range(128, len(trX) + 1, 128)):
            sess.run(train_op, feed_dict={X: trX[start: end], Y: trY[start: end], p_keep_input: 0.8, p_keep_hidden: 0.5})
        # 更新计数器
        global_step.assign(i).eval()
        # 储存模型
        saver.save(sess, ckpt_dir + "/model.ckpt", global_step=global_step)

Start from:  0
